In [1]:
#import required libraires
import pandas as pd
import numpy as np
from langdetect import detect
import reverse_geocode

In [2]:
df= pd.read_csv('Hotel_Reviews.csv')
df.head(3)

,Hotel_Address,Additional_Number_of_Scoring,Review_Date,Average_Score,Hotel_Name,Reviewer_Nationality,Negative_Review,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,Positive_Review,Review_Total_Positive_Word_Counts,Total_Number_of_Reviews_Reviewer_Has_Given,Reviewer_Score,Tags,days_since_review,lat,lng
0,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,8/3/2017,7.7,Hotel Arena,Russia,I am so angry that i made this post available...,397,1403,Only the park outside of the hotel was beauti...,11,7,2.9,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968
1,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,8/3/2017,7.7,Hotel Arena,Ireland,No Negative,0,1403,No real complaints the hotel was great great ...,105,7,7.5,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968
2,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/31/2017,7.7,Hotel Arena,Australia,Rooms are nice but for elderly a bit difficul...,42,1403,Location was good and staff were ok It is cut...,21,9,7.1,"[' Leisure trip ', ' Family with young childre...",3 days,52.360576,4.915968


In [3]:
df.columns=[x.lower() for x in df.columns]

In [4]:
df.columns

Index(['hotel_address', 'additional_number_of_scoring', 'review_date',
       'average_score', 'hotel_name', 'reviewer_nationality',
       'negative_review', 'review_total_negative_word_counts',
       'total_number_of_reviews', 'positive_review',
       'review_total_positive_word_counts',
       'total_number_of_reviews_reviewer_has_given', 'reviewer_score', 'tags',
       'days_since_review', 'lat', 'lng'],
      dtype='object')

In [5]:
#there are 1492 hotels
hotel=list(df.hotel_name.unique())
len(hotel)

1492

In [6]:
df[df['hotel_name'].str.contains("H tel")]['hotel_name'].unique()[:10]

array(['H tel De Vend me', 'H tel des Ducs D Anjou',
       'H tel Juliana Paris', 'H tel de Jos phine BONAPARTE',
       'H tel Keppler', 'H tel Chaplain Paris Rive Gauche',
       'H tel Regina Op ra Grands Boulevards', 'H tel Diva Opera',
       'H tel Duo', 'H tel Le Marianne'], dtype=object)

In [7]:
df['hotel_name']=df['hotel_name'].apply(lambda x:x.replace('H tel','Hotel'))

In [8]:
# sanity check to check
df[df.index==12608]['hotel_name']

12608    Hotel De Vend me
Name: hotel_name, dtype: object

In [9]:
#group by hotelname and aggregate
geocode_df = df.groupby('hotel_name').agg({'lat': 'first','lng':'first'}).reset_index()

In [10]:
#check if there is null
geocode_df.isnull().sum()

hotel_name     0
lat           17
lng           17
dtype: int64

In [11]:
#drop null and reset index
geocode_df.dropna(subset=['lat','lng'],inplace=True)
geocode_df.reset_index(drop='index',inplace=True)

In [12]:
#sanity check if there is null
geocode_df.isnull().sum()

hotel_name    0
lat           0
lng           0
dtype: int64

In [13]:
#create new column
geocode_df['location']= ''
        

In [14]:
def search(x):
    for i in range (x.shape[0]):
        co= ((geocode_df['lat'][i], geocode_df['lng'][i]),)
        geocode_df['location'][i]= reverse_geocode.search(co)
search(geocode_df)

C:\Users\WAQARC~1\AppData\Local\Temp/ipykernel_6344/2784232231.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  geocode_df['location'][i]= reverse_geocode.search(co)


In [15]:
geocode_df.head(10)

,hotel_name,lat,lng,location
0,11 Cadogan Gardens,51.493616,-0.159235,"[{'country_code': 'GB', 'city': 'Chelsea', 'co..."
1,1K Hotel,48.863932,2.365874,"[{'country_code': 'FR', 'city': 'Paris', 'coun..."
2,25hours Hotel beim MuseumsQuartier,48.206474,16.354630,"[{'country_code': 'AT', 'city': 'Vienna', 'cou..."
3,41,51.498147,-0.143649,"[{'country_code': 'GB', 'city': 'West End of L..."
4,45 Park Lane Dorchester Collection,51.506371,-0.151536,"[{'country_code': 'GB', 'city': 'West End of L..."
5,88 Studios,51.499279,-0.209073,"[{'country_code': 'GB', 'city': 'Kensington', ..."
6,9Hotel Republique,48.870842,2.360586,"[{'country_code': 'FR', 'city': 'Paris', 'coun..."
7,A La Villa Madame,48.848861,2.331526,"[{'country_code': 'FR', 'city': 'Paris', 'coun..."
8,ABaC Restaurant Hotel Barcelona GL Monumento,41.410694,2.136294,"[{'country_code': 'ES', 'city': 'La Bonanova',..."
9,AC Hotel Barcelona Forum a Marriott Lifestyle ...,41.410131,2.218805,"[{'country_code': 'ES', 'city': 'Diagonal Mar'..."


In [16]:

#function to search for country
def search_country(x):
    return x[0]['country']

In [17]:
#map the city
geocode_df['country']=geocode_df['location'].map(search_country)

In [18]:
geocode_df.country.unique()

array(['United Kingdom', 'France', 'Austria', 'Spain', 'Italy',
       'Netherlands'], dtype=object)

In [19]:
#function to search for city
def search_city(x):
    return x[0]['city']

In [20]:
#map the city
geocode_df['city']=geocode_df['location'].map(search_city)

In [21]:
geocode_df.city.value_counts()

Paris               224
Vienna              137
Milan               135
Amsterdam            83
Levallois-Perret     82
                   ... 
Buckhurst Hill        1
la Teixonera          1
Sagrada Família       1
Sants                 1
Harringay             1
Name: city, Length: 111, dtype: int64

In [22]:
!pip install folium

In [25]:
from folium import plugins
import folium

In [35]:
#generating a map to output the cities 
map2 = folium.Map(location=[41.3851,2.1734], zoom_start=4)
folium.raster_layers.TileLayer('Open Street Map').add_to(map2)
for la,lo in zip(geocode_df.lat,geocode_df.lng):
    folium.Marker(
        location=[la,lo],
        icon=folium.Icon(icon_color='white')
    ).add_to(map2)
# Plotting 
map2.save('./images/Europe_overview.html')
map2

In [36]:
#save to pickle
geocode_df.to_pickle('./data/geocode.pkl')

In [37]:
tag_df=df[['hotel_name','tags','lat','lng']]

In [38]:
tag_df.head()

,hotel_name,tags,lat,lng
0,Hotel Arena,"[' Leisure trip ', ' Couple ', ' Duplex Double...",52.360576,4.915968
1,Hotel Arena,"[' Leisure trip ', ' Couple ', ' Duplex Double...",52.360576,4.915968
2,Hotel Arena,"[' Leisure trip ', ' Family with young childre...",52.360576,4.915968
3,Hotel Arena,"[' Leisure trip ', ' Solo traveler ', ' Duplex...",52.360576,4.915968
4,Hotel Arena,"[' Leisure trip ', ' Couple ', ' Suite ', ' St...",52.360576,4.915968


In [39]:
tag_df = tag_df.groupby('hotel_name').agg({'tags': ', '.join,'lat':'first','lng':'first'}).reset_index()

In [40]:
tag_df.shape

(1491, 4)

In [41]:
tag_df.isnull().sum()

hotel_name     0
tags           0
lat           17
lng           17
dtype: int64

In [42]:
tag_df.head()

,hotel_name,tags,lat,lng
0,11 Cadogan Gardens,"[' Leisure trip ', ' Couple ', ' Superior Quee...",51.493616,-0.159235
1,1K Hotel,"[' Leisure trip ', ' Couple ', ' Superior M Do...",48.863932,2.365874
2,25hours Hotel beim MuseumsQuartier,"[' Leisure trip ', ' Solo traveler ', ' Standa...",48.206474,16.354630
3,41,"[' Leisure trip ', ' Couple ', ' Executive Kin...",51.498147,-0.143649
4,45 Park Lane Dorchester Collection,"[' Leisure trip ', ' Solo traveler ', ' Execut...",51.506371,-0.151536


In [43]:
tag_df.dropna(subset=['lat','lng'],inplace=True)

In [44]:
tag_df.to_pickle("./data/hoteltag.pkl")

In [45]:
#suppose to be empty 
df[df['positive_review']=='No Positive'][['negative_review','positive_review']].head(5)

,negative_review,positive_review
8,Even though the pictures show very clean room...,No Positive
32,Our bathroom had an urine order Shower was ve...,No Positive
98,Got charged 50 for a birthday package when it...,No Positive
121,The first room had steep steps to a loft bed ...,No Positive
134,Foyer was a mess Only place to relax was the ...,No Positive


In [46]:
# suppose to be empty 

df[df['negative_review']=='No Negative'][['negative_review','positive_review']].head(5)

,negative_review,positive_review
1,No Negative,No real complaints the hotel was great great ...
13,No Negative,This hotel is being renovated with great care...
15,No Negative,This hotel is awesome I took it sincirely bec...
18,No Negative,Public areas are lovely and the room was nice...
48,No Negative,The quality of the hotel was brilliant and ev...


In [47]:
#replace No negative and no positive
df["negative_review"] = df["negative_review"].apply(lambda x: x.replace("No Negative", "No"))
df["positive_review"] = df["positive_review"].apply(lambda x: x.replace("No Positive", "No"))


In [48]:
#function to check negative & positive duplicates
def check(x):
    pos, neg = x
    if pos ==  neg:
        return 1
    return 0

In [49]:
df['check_dup'] = [check(x) for x in df[['positive_review','negative_review']].values]
index_col= df[df['check_dup']==1].index

In [50]:
df[df['check_dup']==1][['positive_review','negative_review']].head()

,positive_review,negative_review
1403,No,No
2451,The hotel good location and clean but some st...,The hotel good location and clean but some st...
2872,Ok,Ok
3067,Standard Hotel,Standard Hotel
5839,I was completely disappointed and mad since t...,I was completely disappointed and mad since t...


In [51]:
for x in index_col:
    df['positive_review'][x]='nothing'

C:\Users\WAQARC~1\AppData\Local\Temp/ipykernel_6344/3929709946.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['positive_review'][x]='nothing'


In [52]:
#dedupe duplicates
df[df['check_dup']==1][['positive_review','negative_review']].head()

,positive_review,negative_review
1403,nothing,No
2451,nothing,The hotel good location and clean but some st...
2872,nothing,Ok
3067,nothing,Standard Hotel
5839,nothing,I was completely disappointed and mad since t...


In [53]:
#this is to group by hotel name and joining the reviews into on observation with ','.
df1 = df.groupby('hotel_name').agg({'negative_review': ', '.join,'positive_review': ', '.join,
                                    'lat': 'first','lng':'first','hotel_address':'first',
                                    'tags': ', '.join}).reset_index()

In [54]:
#check if there is null
df1.isnull().sum()

hotel_name          0
negative_review     0
positive_review     0
lat                17
lng                17
hotel_address       0
tags                0
dtype: int64

In [55]:
df1.dropna(subset=['lat','lng'],inplace=True)

In [56]:
#import pickle with city
geocode=pd.read_pickle('./data/geocode.pkl')

In [57]:
# merging the dataset based on hotel name
new_df= pd.merge(df1,geocode,on='hotel_name',how='outer')

In [58]:
new_df

,hotel_name,negative_review,positive_review,lat_x,lng_x,hotel_address,tags,lat_y,lng_y,location,country,city
0,11 Cadogan Gardens,Thought the prise of drinks at the bar a litt...,We were particularly impressed by the very wa...,51.493616,-0.159235,11 Cadogan Gardens Sloane Square Kensington an...,"[' Leisure trip ', ' Couple ', ' Superior Quee...",51.493616,-0.159235,"[{'country_code': 'GB', 'city': 'Chelsea', 'co...",United Kingdom,Chelsea
1,1K Hotel,Air conditioning in room didn t work and desp...,Location good close to le Marais and 3e arron...,48.863932,2.365874,13 Boulevard Du Temple 3rd arr 75003 Paris France,"[' Leisure trip ', ' Couple ', ' Superior M Do...",48.863932,2.365874,"[{'country_code': 'FR', 'city': 'Paris', 'coun...",France,Paris
2,25hours Hotel beim MuseumsQuartier,Breakfast not included and buffet really expe...,Cool vintage style in the middle of the museu...,48.206474,16.354630,Lerchenfelder Stra e 1 3 07 Neubau 1070 Vienna...,"[' Leisure trip ', ' Solo traveler ', ' Standa...",48.206474,16.354630,"[{'country_code': 'AT', 'city': 'Vienna', 'cou...",Austria,Vienna
3,41,"There wasn t a thing that we didn t like , No...",Its central proximity close to all services a...,51.498147,-0.143649,41 Buckingham Palace Road Westminster Borough ...,"[' Leisure trip ', ' Couple ', ' Executive Kin...",51.498147,-0.143649,"[{'country_code': 'GB', 'city': 'West End of L...",United Kingdom,West End of London
4,45 Park Lane Dorchester Collection,More kinds of fruit juice will make the mini ...,Everything here are almost perfect the staffs...,51.506371,-0.151536,45 Park Lane Westminster Borough London W1K 1P...,"[' Leisure trip ', ' Solo traveler ', ' Execut...",51.506371,-0.151536,"[{'country_code': 'GB', 'city': 'West End of L...",United Kingdom,West End of London
...,...,...,...,...,...,...,...,...,...,...,...,...
1469,citizenM London Bankside,This was our third stay at this hotel and it ...,"No, What an amazing and unique hotel A short ...",51.505151,-0.100472,20 Lavington Street Southwark London SE1 0NZ U...,"[' Leisure trip ', ' Couple ', ' Double Room '...",51.505151,-0.100472,"[{'country_code': 'GB', 'city': 'City of Londo...",United Kingdom,City of London
1470,citizenM London Shoreditch,Lifts need reprogramming exasperating journey...,Swift auto check in Good bar and lounge area ...,51.524137,-0.078698,6 Holywell Lane Hackney London EC2A 3ET United...,"[' Leisure trip ', ' Solo traveler ', ' Double...",51.524137,-0.078698,"[{'country_code': 'GB', 'city': 'Barbican', 'c...",United Kingdom,Barbican
1471,citizenM Tower of London,"Rooms are small but well designed, breakfast...",The use of technology was impressive UBS outl...,51.510237,-0.076443,40 trinity square City of London London EC3N 4...,"[' Leisure trip ', ' Solo traveler ', ' Double...",51.510237,-0.076443,"[{'country_code': 'GB', 'city': 'City of Londo...",United Kingdom,City of London
1472,every hotel Piccadilly,The hotel overall requires an update furnitur...,The location was the only great aspect of thi...,51.510146,-0.131506,39 Coventry Street Westminster Borough London ...,"[' Leisure trip ', ' Couple ', ' Standard Doub...",51.510146,-0.131506,"[{'country_code': 'GB', 'city': 'London', 'cou...",United Kingdom,London


In [59]:
# exporting to pickle
new_df.to_pickle('./data/review.pkl')